# B6: $^{13}$C MFA for microbial communities

# Introduction

This notebook reproduces the results obtained by an excel notebook provided by Paul Djkistra (personal communication) and used to obtain the fits to $CO_2$ labeling and the  Carbon Use Efficiency (CUE) results in Hagerty *et al* 2014. The mathematical framework is explained in Djikstra *et al* 2011.

# Setup

First, we need to set the path and environment variable properly:

In [ ]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [ ]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

dirFiles = os.environ["QUANTMODELPATH"]+'/data/tests/Hagerty2014/'

Then, let's load all needed python packages:

In [ ]:
%matplotlib inline
import unittest, os, sys
from IPython.display import SVG
import ToyaData as TD
import shelve
import enhancedLists, GAMSclasses, FluxModels

And move to a scratch directory (to make things faster):

# General functions for input file generation

In this section, we define the functions needed to create the input files for the library.

This first function creates input files for solving the Djikstra problem, which is not a standard $^{13}$C MFA problem:

In [ ]:
def DijkstraInputFiles2(transitionsFile1,transitionsFile2,feedAFile,feedBFile,feedCFile,feedDFile,fluxLimitsFile,ratios):
    # Load carbon transitions 
    atomTransitionsA = enhancedLists.AtomTransitionList(transitionsFile1)
    atomTransitionsB = enhancedLists.AtomTransitionList(transitionsFile1)
    atomTransitionsC = enhancedLists.AtomTransitionList(transitionsFile2)
    atomTransitionsD = enhancedLists.AtomTransitionList(transitionsFile2)
    
    # Add both feeds
    ReacNetA = atomTransitionsA.getReactionNetwork('E. coli wt5h 13C MFA')
    ReacNetA.addFeed(feedAFile)
    ReacNetA.loadFluxBounds(fluxLimitsFile,measured=True)
    ReacNetB = atomTransitionsB.getReactionNetwork('E. coli wt5h 13C MFA')
    ReacNetB.addFeed(feedBFile)
    ReacNetC = atomTransitionsC.getReactionNetwork('E. coli wt5h 13C MFA')
    ReacNetC.addFeed(feedCFile)
    ReacNetD = atomTransitionsD.getReactionNetwork('E. coli wt5h 13C MFA')
    ReacNetD.addFeed(feedDFile)
    
    # Obtain input files from first feed
    inputFiles = []
    filesA = ReacNetA.getEMUfiles()
    filesA.extend(ReacNetA.getStoichMetRxn13CFiles())
    filesA.extend(ReacNetA.getSourceLabelFile())
    
    # Obtain source labeling file for second feed (all others are the same)
    filesB = ReacNetB.getSourceLabelFile()
    
    # Obtain source labeling file for third and fourth feed
    filesC = ReacNetC.getSourceLabelFile()
    filesD = ReacNetD.getSourceLabelFile()
    
    # Obtain second emm file and change to EMM2
    emm2files = ReacNetC.getEMUfiles()
    for name,string in emm2files:
        if 'emm_simplified' in name:
            name   = name.replace("emm_simplified","emm_simplified2")
            string = string.replace("EMM","EMM2")
            inputFiles.append((name,string))
            
    # Obtain second set of metabolites
    stoich2files = ReacNetC.getStoichMetRxn13CFiles()
    for name,string in stoich2files:
        if 'StoichNetwork' in name:
            name = name.replace('StoichNetwork','StoichNetwork2')
            string = string.replace("S(","S2(")
            inputFiles.append((name,string))

    # Change source labeling file for first feed 
    for name,string in filesA:
        if 'Source_Labeling' not in name:
            inputFiles.append((name,string))
        else:
            newName = 'Source_LabelingA.txt'
            string = string.replace("f.fx","fA.fx")
            inputFiles.append((newName,string))
    
    # Change source labeling file for others feed
    
    for fileList, tag in zip([filesB,filesC,filesD],['B','C','D']):
        for name,string in fileList:
            if 'Source_Labeling' in name:
                newName = "Source_Labeling"+str(tag)+".txt"
                string = string.replace("f.fx","f"+str(tag)+".fx")
                inputFiles.append((newName,string))
    
    # Extra files that are needed 
    twelveset = ['0','1','2','3','4','5','6','7','8','9','10','11','12']
    mset      =  GAMSclasses.GAMSSet('mset',set(twelveset))
    nset      =  GAMSclasses.GAMSSet('nset',set(twelveset))

    msetfilename = 'mset.txt'
    msetSt = mset.write('toString')
    nsetfilename = 'nset.txt'
    nsetSt = nset.write('toString')

    inputFiles.append((msetfilename,msetSt))
    inputFiles.append((nsetfilename,nsetSt))
    
    # Add glucose and pyruvate ratios
    ratioFile = 'ratios.txt'
    lines = []
    ratioGlc = ratios[0]
    ratioPyr = ratios[1]
    
    lines.append('ratioGlu = '+str(ratioGlc)+';')
    lines.append('ratioPyr = '+str(ratioPyr)+';')        
    
    inputFiles.append((ratioFile,'\n'.join(lines)))
    
    return inputFiles

This function is used to automatically create flux bounds files:

In [ ]:
def createFluxBoundsFile(r10s,r14s):
    boundsFile = open('fluxBounds.txt','w')
    boundsFile.write('r10:	'+str(r10s[0])+' [==] '+str(r10s[1])+'\n')
    boundsFile.write('r14:	'+str(r14s[0])+' [==] '+str(r14s[1])+'\n')
    boundsFile.close()

This function finds the CUE (Carbon Use Efficieny) as per equation 12 in Dijkstra *et al* Soil Biology and Biochemistry 2011:

In [ ]:
def findCUE(resultsDict):
    CUEfluxes = [5,7,8,10]
    co2Flux = 0
    for key in sorted(resultsDict['Vout'].elements.keys(),key=lambda rname: int(rname[0].replace('r','')) ):
        name = key[0]
        fluxNumber = int(name.replace('r',''))
        if fluxNumber in CUEfluxes:
            co2Flux += resultsDict['Vout'].elements[key]
            
    CUE = (600 - co2Flux)/600
    
    return CUE

This function takes the labeling ratios and does the full calculation to provide the CUE: 

In [ ]:
def ratio2CUE(ratios):
    # Get input Files
    createFluxBoundsFile([0,100],[0,100])
    inputFiles = DijkstraInputFiles2(dirFiles+'REACTIONSDijkstraGlu.txt',dirFiles+'REACTIONSDijkstraPyr.txt',
                                     dirFiles+'FEEDgluA.txt',dirFiles+'FEEDgluB.txt',
                                     dirFiles+'FEEDpyrA.txt',dirFiles+'FEEDpyrB.txt',dirFiles+'fluxBounds.txt',
                                     ratios)    
    
    # Create GAMS problem
    GAMSfileName = dirFiles+'Hagerty.gms'   # The gams files contains all the problem formulation as
                                            # per Dijkstra et al 2011
    outputFuncs = { 'fAout':(GAMSclasses.GAMSParameterFromResult,['fAout.txt','fAout']),
                    'fBout':(GAMSclasses.GAMSParameterFromResult,['fBout.txt','fBout']),
                    'fCout':(GAMSclasses.GAMSParameterFromResult,['fCout.txt','fBout']),
                    'fDout':(GAMSclasses.GAMSParameterFromResult,['fDout.txt','fBout']),                
                    'Vout' :(FluxModels.fluxParOut,['Vout.txt','Vout']),
                    'Info' :(FluxModels.infoOut,['OFGenInfo.txt']),                
                        }
    GAMSInputFiles = inputFiles

    GAMSprob       = GAMSclasses.GAMSProblem('Djkistra',GAMSfileName,GAMSInputFiles,outputFuncs,execType='serial')  
    GAMSprob.erase = True
    
    # Solve GAMS problem
    with GAMSprob as prob:
        # Run problem
        prob.run()

        # Check if it is done
        prob.waitTilDone()

        # Collect data
        prob.collect()
        
        # Output data 
        resultsDict = prob.Results 
        
    # Find CUE and rest of output
    CUE = findCUE(resultsDict)
    
    fitRatioGlu = resultsDict['fAout'].elements[('co2_c_1', '1')]/resultsDict['fBout'].elements[('co2_c_1', '1')]
    fitRatioPyr = resultsDict['fCout'].elements[('co2_c_1', '1')]/resultsDict['fDout'].elements[('co2_c_1', '1')]
    OF = resultsDict['Info'].OF
    solve = resultsDict['Info'].solvestat
    model = resultsDict['Info'].modelstat
    
    return CUE, [fitRatioGlu,fitRatioPyr], [OF, solve, model]

# Application to Hagerty *et al* 2014 data

## Create input files

Creating input files:

In [ ]:
createFluxBoundsFile([0,100],[0,100])
ratios = [2.48,2.00]

inputFiles = DijkstraInputFiles2(dirFiles+'REACTIONSDijkstraGlu.txt',dirFiles+'REACTIONSDijkstraPyr.txt',
                                 dirFiles+'FEEDgluA.txt',dirFiles+'FEEDgluB.txt',
                                 dirFiles+'FEEDpyrA.txt',dirFiles+'FEEDpyrB.txt',dirFiles+'fluxBounds.txt',
                                 ratios)

## Create GAMS problem

Producing optimization problem in GAMS mode:

In [ ]:
GAMSfileName = dirFiles+'Hagerty.gms'
outputFuncs = { 'fAout':(GAMSclasses.GAMSParameterFromResult,['fAout.txt','fAout']),
                'fBout':(GAMSclasses.GAMSParameterFromResult,['fBout.txt','fBout']),
                'fCout':(GAMSclasses.GAMSParameterFromResult,['fCout.txt','fBout']),
                'fDout':(GAMSclasses.GAMSParameterFromResult,['fDout.txt','fBout']),                
                'Vout' :(FluxModels.fluxParOut,['Vout.txt','Vout']),
                'Info' :(FluxModels.infoOut,['OFGenInfo.txt']),                
                        }
GAMSInputFiles = inputFiles

In [ ]:
GAMSprob       = GAMSclasses.GAMSProblem('Djkistra',GAMSfileName,GAMSInputFiles,outputFuncs,execType='serial')  
GAMSprob.erase = False

## Solve GAMS problem

In [ ]:
cd /scratch/hgmartin_scratch/tests